## This Notebook selects and generates the file for severe weather events we care about

#### Download data files

2013 data: 
`https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2013_c20170419.csv.gz`

2014 data: 
`StormEvents_details-ftp_v1.0_d2014_c20161118.csv.gz`

In [2]:
url_2013 = 'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2013_c20170419.csv.gz'
url_2014 = 'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2014_c20161118.csv.gz'
url_2015 = 'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2015_c20170216.csv.gz'
url_2016 = 'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2016_c20170419.csv.gz'

import os
from six.moves.urllib.request import urlretrieve

if not os.path.exists('StormEvents_2013.csv.gz'):
    urlretrieve(url_2013, 'StormEvents_2013.csv.gz')
if not os.path.exists('StormEvents_2014.csv.gz'):
    urlretrieve(url_2014, 'StormEvents_2014.csv.gz')
if not os.path.exists('StormEvents_2015.csv.gz'):
    urlretrieve(url_2015, 'StormEvents_2015.csv.gz')
if not os.path.exists('StormEvents_2016.csv.gz'):
    urlretrieve(url_2016, 'StormEvents_2016.csv.gz')

Then, save the unzipped files as `StormEvents_2013.csv` and `StormEvents_2014.csv`

In [3]:
import gzip

if not os.path.exists('StormEvents_2013.csv'):
    gz = gzip.open('StormEvents_2013.csv.gz')
    gz_out = open('StormEvents_2013.csv', 'w')
    gz_out.write( gz.read() )
    gz_out.close()
    gz.close()

if not os.path.exists('StormEvents_2014.csv'):
    gz = gzip.open('StormEvents_2014.csv.gz')
    gz_out = open('StormEvents_2014.csv', 'w')
    gz_out.write( gz.read() )
    gz_out.close()
    gz.close()
 
if not os.path.exists('StormEvents_2015.csv'):
    gz = gzip.open('StormEvents_2015.csv.gz')
    gz_out = open('StormEvents_2015.csv', 'w')
    gz_out.write( gz.read() )
    gz_out.close()
    gz.close()

if not os.path.exists('StormEvents_2016.csv'):
    gz = gzip.open('StormEvents_2016.csv.gz')
    gz_out = open('StormEvents_2016.csv', 'w')
    gz_out.write( gz.read() )
    gz_out.close()
    gz.close()

Read the location of every county, so we can covert back from geolocation to state name and county name.

In [4]:
import pandas as pd

states_short2long = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

zip_code_file = 'zip_codes_states.csv'
zip_codes = pd.read_csv(zip_code_file)
county_geo_list = []
county_name_list = []
for row in zip_codes.iterrows():
    entry = row[1]
    lat = entry.latitude
    lng = entry.longitude
    county = entry.county
    state = entry.state
    if (not (pd.isnull(lat) or pd.isnull(lng) or pd.isnull(county) or pd.isnull(state))
        and state in states_short2long):
        county_geo_list.append((lat, lng))
        county_name_list.append((county, states_short2long[state]))

However, counties in this file are fine-grained. So we will use the features file to get the list of counties that have weather data collected instead.

In [6]:
features_file = pd.read_csv('feature_data_2.csv')
counties = set()

for row in features_file.iterrows():
    county = row[1]['County.Name']
    state = row[1]['State.Name']
    counties.add( (county, state) )
    
county_name_list = list(counties)
county_geo_list = []

for county, state in county_name_list:
    average_lat = features_file.loc[(features_file['County.Name'] == county) &
                                   (features_file['State.Name'] == state)]['Latitude'].mean()
    average_lng = features_file.loc[(features_file['County.Name'] == county) &
                                   (features_file['State.Name'] == state)]['Longitude'].mean()
    county_geo_list.append( (average_lat, average_lng) )

In [7]:
from scipy import spatial

county_tree = spatial.KDTree(county_geo_list)

Select entries in a square region within a certain time frame

In [8]:
data_2013 = pd.read_csv('StormEvents_2013.csv')
data_2014 = pd.read_csv('StormEvents_2014.csv')
data_2015 = pd.read_csv('StormEvents_2015.csv')
data_2016 = pd.read_csv('StormEvents_2016.csv')
orig_data = pd.concat([data_2013, data_2014, data_2015, data_2016])

min_lat = 39.66
max_lat = 45.41
min_lng = -93.64
max_lng = -86.00
start_mon = 201601
end_mon = 201612

out_data = pd.DataFrame(columns=['date', 'type', 'lat', 'lng', 'county', 'state'])

for row in orig_data.iterrows():
    entry = row[1]
    if ((not pd.isnull(entry.BEGIN_YEARMONTH) and start_mon <= int(entry.BEGIN_YEARMONTH) <= end_mon) and
        (not pd.isnull(entry.BEGIN_LAT) and min_lat < entry.BEGIN_LAT < max_lat) and
        (not pd.isnull(entry.BEGIN_LON) and min_lng < entry.BEGIN_LON < max_lng) ):
        county_name, state_name = county_name_list[
                county_tree.query((entry.BEGIN_LAT, entry.BEGIN_LON))[1]]
        new_record = pd.DataFrame([[str(int(entry.BEGIN_YEARMONTH)) + str(int(entry.BEGIN_DAY)), 
                entry.EVENT_TYPE, entry.BEGIN_LAT, entry.BEGIN_LON,
                county_name, state_name
                ]],
                columns=['date', 'type', 'lat', 'lng', 'county', 'state'])
        # print new_record
        out_data = out_data.append(new_record, ignore_index=True)

In [9]:
print out_data

          date               type      lat      lng         county      state
0     20160315               Hail  42.8400 -89.0700           Dane  Wisconsin
1     20160315          Lightning  43.1069 -88.3468       Waukesha  Wisconsin
2      2016075            Tornado  44.3935 -92.3456      La Crosse  Wisconsin
3      2016077  Thunderstorm Wind  43.2800 -92.8100     Black Hawk       Iowa
4      2016075  Thunderstorm Wind  44.1900 -92.6500      La Crosse  Wisconsin
5     20160723        Flash Flood  42.0933 -87.7221           Cook   Illinois
6     20161118  Thunderstorm Wind  41.0500 -86.6000        Carroll    Indiana
7      2016075  Thunderstorm Wind  45.0600 -93.3500        Douglas  Wisconsin
8      2016075  Thunderstorm Wind  44.8300 -93.4600      Palo Alto       Iowa
9      2016075  Thunderstorm Wind  44.9400 -93.0600        Douglas  Wisconsin
10    20160728          Lightning  41.6700 -88.1300           Cook   Illinois
11     2016075  Thunderstorm Wind  44.6834 -92.3715      La Cros

The features file has every entry with lat, lng, and county, state names. So we may build a dict of the locations of each county, then match the events to counties.

In [12]:
grouped = out_data.groupby(['date', 'type', 'county', 'state'], as_index=False).mean()
grouped.to_csv('selected.csv')

In [13]:
print grouped

         date                      type         county      state        lat  \
0    20160111                     Flood      Van Buren       Iowa  40.975000   
1    20160117                     Flood     Black Hawk       Iowa  42.560000   
2     2016017                     Flood      Van Buren       Iowa  40.975000   
3    20160219                     Flood          Grant  Wisconsin  42.722500   
4    20160220                     Flood     Black Hawk       Iowa  42.480000   
5    20160221                     Flood      Van Buren       Iowa  40.870000   
6    20160222                     Flood      Van Buren       Iowa  41.080000   
7    20160223                     Flood           Polk       Iowa  41.590000   
8    20160311                     Flood           Sauk  Wisconsin  44.026500   
9    20160315                      Hail          Clark   Illinois  40.156000   
10   20160315                      Hail        Clinton       Iowa  41.294573   
11   20160315                      Hail 